In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt   
from pydotplus import graph_from_dot_data
from IPython.display import Image  
import seaborn as sns
from IPython.display import HTML, display
import tabulate

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#reading datafiles
#df1 = pd.read_csv('data/2014 excel.csv',na_values=[" ","NA","NaN",""])
#df2 = pd.read_csv('data/2015 excel.csv',na_values=[" ","NA","NaN",""])
result = pd.read_csv('data/2016 excel.csv',na_values=[" ","NA","NaN",""])

#merging datafiles
#frames = [df1, df2, df3]
#result = pd.concat(frames, axis=0)
result.isnull().sum() 

In [ ]:
#dropping unnecessary columns
result= result.drop(['Grid Ref: Northing','Grid Ref: Easting','Reference Number','Expr1','Accident Date','Day','Time (24hr)','1st Road Class'],axis=1)

#dropping observations with missing values
result= result.dropna(axis=0)

In [ ]:
result.head(3)

In [ ]:
result.columns

In [ ]:
result["Road Surface"].value_counts()

In [ ]:
result["Number of Vehicles"].value_counts()

In [ ]:
result["Lighting Conditions"].value_counts()

In [ ]:
result["Weather Conditions"].value_counts()

In [ ]:
result["Casualty Class"].value_counts()

In [ ]:
result["Casualty Severity"].value_counts()          

In [ ]:
result["Sex of Casualty"].value_counts()

In [ ]:
result["Type of Vehicle"].value_counts()

In [ ]:
cleanup_nums = {"Road Surface":     {"Dry": 1, "Wet or Damp": 2, "Snow": 3,"Frost or Ice": 3, "Flood (surface water over 3cm deep)": 4},
                "Lighting Conditions": {"Daylight street lights present": 1, "Darkness street lights present and lit": 2, "Darkness no street lighting": 3, 
                                  "Darkness street lighting unknown": 3, "Darkness street lights present but unlit": 3},
               "Weather Conditions":     {"Fine without high winds": 1, "Raining without high winds": 2, "Unknown": 5, 
                                  "Raining with high winds": 3, "Fine with high winds": 1,"Other": 5,"Snowing without high winds": 4,
                                          "Fog or mist ": 4,"Snowing with high winds": 4, "Darkness: street lighting unknown": 5},
               "Casualty Class":     {"Driver": 1, "Passenger": 2,"Driver  ": 1,"Pedestrian": 3},
                "Casualty Severity":     {"Slight": 1, "Serious": 2,"Fatal": 3},
               "Sex of Casualty":     {"Male": 1, "Female": 2},
                 "Type of Vehicle":     {"Car": 1, "Pedal cycle": 2,"Bus or coach (17 or more passenger seats)": 3,
                                         "Taxi or Private hire car": 1,
                                      "Motorcycle over 50cc and up to 125cc": 2, "Goods vehicle 3.5 tonnes mgw and under": 3,
                                         "Motorcycle over 500cc" : 2,"M cycle 50cc and under": 2,
                                      "Motorcycle over 125cc and up to 500cc": 2, "Goods vehicle 7.5 tonnes mgw and over": 3,
                                         "Other Vehicle ": 4,"Minibus (8 to 16 passenger seats)": 3,
                         "Mobility Scooter" : 2, "Goods vehicle over 3.5 tonnes and under 7.5 tonnes mgw": 3,
                                         "Agricultural vehicle (includes diggers etc.)": 3,"Minibus (8  to 16 passenger seats)": 3,
                                      "Motorcycle Unknown CC": 2, "Tram or Light rail": 4,"Ridden horse": 4}}

In [ ]:
result.replace(cleanup_nums, inplace=True)
result.head()

In [ ]:
result.shape

In [ ]:
sorted(result["Weather Conditions"].unique())

In [ ]:
result.isnull().sum() 

In [ ]:
result.corr()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot()

In [ ]:
#result.plot("Casualty Severity", kind="bar")

In [ ]:
result['Casualty Severity'].value_counts()

In [ ]:
#dividing dataset into two: X- independent variables and y- dependent variable  
X = result.drop('Casualty Severity', axis=1)
y = result['Casualty Severity']

In [ ]:
#defining the model with max_depth=6
tree = DecisionTreeClassifier(criterion='gini',max_depth=7,random_state=1)  
tree.fit(X, y) 
y_pred=tree.predict(X)

#evaluation procedure
kfold = KFold(n_splits=10, shuffle=True, random_state=1)

#cross validation
score = cross_val_score(tree, X, y, cv=kfold)
print("mean accuracy %.2f%% (standard deviation %.2f%%)" % (score.mean()*100, score.std()*100))
#The results are summarized as both the mean and standard deviation of the model accuracy on the dataset.

In [ ]:
result.hist(bins=15, figsize=(20, 6), layout=(2, 5));

In [ ]:
f, ax = plt.subplots(nrows=1,ncols=3,figsize=(20, 8)) 
for i,j in zip(X.columns.tolist(), ax.flatten()):
    sns.countplot(x=X[i],ax=j)  

In [ ]:
corr = result.corr()
sns.heatmap(corr)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
ag_cluster= AgglomerativeClustering(n_clusters=3)

In [ ]:
import sklearn.linear_model as lm
lr =lm.LassoLars()
Y_ =result['Casualty Severity']
ind_var = result.columns
ind_vars = ind_var.delete(5)
X_ = result[ind_vars]
ind_vars

In [ ]:
lr.fit(X_,Y_)

In [ ]:
print("Coefficient of determination", lr.coef_)

In [ ]:
print("Coefficient path", lr.coef_path_)

In [ ]:
print("Intercept", lr.intercept_)

In [ ]:
print("Alphas", lr.alphas_)

In [ ]:
print("Number of iterations", lr.n_iter_)

In [ ]:
Y_pred = lr.predict(X)
result["Predicted Casualty Severity"] = Y_pred
result.head(3)

In [ ]:
plt.scatter(result["Age of Casualty"],result["Casualty Severity"], color= "green")
plt.scatter(result["Age of Casualty"],result["Predicted Casualty Severity"], color= "red")
plt.title("Actual vs Predicted- based on Age of Casualty")
plt.xlabel("Age of victim")
plt.ylabel("Casualty Severity")

In [ ]:
accident_case={}
for feature in ind_vars:
    temp=input("Enter "+feature+": ")
    accident_case[feature]=temp
user_case=pd.DataFrame(data=accident_case, index=[0], columns=ind_vars)
casualty= lr.predict(user_case)
print("Casualty Severity of the victim is", int(casualty[3]))